In [6]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import keyboard
from datetime import datetime

In [7]:
def baixar_fatura(url, username, password, target_dir, nome, cnpj):
    driver = webdriver.Chrome()
    driver.get(url)
    
    # Preencher o campoRegional com "São Paulo" (Ou estado que desejar)
    driver.find_element(By.ID, 'campoRegional').send_keys('São Paulo')
    
    # Clicar no li com class "ui-menu-item-custom"
    driver.find_element(By.CLASS_NAME, 'ui-menu-item-custom').click()
    
    # Clicar no <a> com class "bt_cliente_s"
    driver.find_element(By.CLASS_NAME, 'bt_cliente_s').click()
    
    # Exemplo de login com os IDs corretos
    driver.find_element(By.ID, 'cpf_cnpj').send_keys(username)
    driver.find_element(By.ID, 'loginPJ_cpfcnpj_button').click()
    time.sleep(3)
    driver.find_element(By.ID, 'senha_fixo').send_keys(password)
    driver.find_element(By.ID, 'loginFixo').click()

    # Verificar se o span com a classe "combo_value variable_value" condiz com o valor da variável cnpj
    time.sleep(15)
    span_element = driver.find_element(By.CLASS_NAME, 'combo_value.variable_value')
    if span_element.text == cnpj:
        pass
    else:
        # Clicar no botão com a classe "btn_combo btn_cnpj"
        driver.find_element(By.CLASS_NAME, 'btn_combo.btn_cnpj').click()
        # Buscar pelo span que possui o data-value igual ao da variável cnpj
        time.sleep(5)
        span_to_select = driver.find_element(By.XPATH, f"//span[@data-value='{cnpj}']")
        # Encontrar o elemento pai <li> do span
        li_element = span_to_select.find_element(By.XPATH, "./ancestor::li")
        # Clicar no elemento <a> dentro do <li> correspondente
        a_element = li_element.find_element(By.TAG_NAME, 'a')
        a_element.click()
    
    # Aciona o botão "botaoSegundaVia"
    iframes = driver.find_elements(By.TAG_NAME, 'iframe')
    for iframe in iframes:
        driver.switch_to.frame(iframe)
        try:
            botao_segunda_via = driver.find_element(By.ID, 'botaoSegundaVia')
            botao_segunda_via.click()
            break
        except:
            print(f'Fatura {nome} está paga')
            return
    
    # Obter o ano e mês atuais
    now = datetime.now()
    ano_atual = now.year
    mes_atual = now.strftime('%m')
    
    # Remover o hífen do CNPJ
    cnpj_sem_hifen = cnpj.replace('-', '')
    
    # Procurar a pasta que contenha os últimos 6 caracteres do CNPJ sem hífen e o nome
    for root, dirs, files in os.walk(target_dir):
        for dir_name in dirs:
            if cnpj_sem_hifen[-6:] in dir_name and nome in dir_name:
                target_dir = os.path.join(root, dir_name)
                break
    
    # Atualizar o target_dir para incluir ano atual, VIVO e mês atual
    target_dir = os.path.join(target_dir, str(ano_atual), 'VIVO', mes_atual)
    
    # Criar os diretórios se não existirem
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    
    # Salvar o arquivo no diretório
    keyboard.press_and_release('ctrl+s')
    time.sleep(1)
    keyboard.write(os.path.join(target_dir, f'{username}.pdf'))
    time.sleep(1)
    keyboard.press_and_release('enter')
    time.sleep(5)
    driver.quit()

    # Mensagem de confirmação
    print(f'Fatura {nome} salva na {target_dir}')

# URL comum para todas as credenciais
url = 'https://login.vivo.com.br/loginmarca/appmanager/marca/publico?acesso=empresas&legacy#'

# Lista de credenciais da fatura que deseja acessar
credenciais = [
    {
        'nome': 'Base 1',
        'username': '22222222222222',
        'password': '111111',
        'target_dir': 'Caminho raiz',
        'cnpj': '2222-22',
        'cnpjFull': '22222222222222'
    },
     {
        'nome': 'Base 2',
        'username': '11111111111111',
        'password': '111111',
        'target_dir': 'Caminho raiz',
        'cnpj': '1111-11',
        'cnpjFull': '11111111111111'
    },
]

# Iterar sobre cada credencial e baixar a fatura
for credencial in credenciais:
    baixar_fatura(url, credencial['username'], credencial['password'], credencial['target_dir'], credencial['nome'], credencial['cnpj'])
   

Fatura Campina Grande está paga
Fatura Goiânia - T4 está paga
Fatura Floripa está paga
Fatura São Leopoldo está paga
Fatura Uberlândia está paga
Fatura SOS Ribeirão Preto está paga
